In [12]:
import pandas as pd
import requests
import json
from pprint import pprint
import os
import errno
from datetime import datetime


In [42]:
province_dict = {
    'Country': None,
    'City': None,
    'City (ASCII)': None,
    'Region': None,
    'Region (ASCII)': None,
    'Population': None,
    'Longitude': None,
    'Latitude': None,
    'Time Zone': None,
}

"""
hourly_data = {
    'Date':None,
    'Year':None,
    'Month': None,
    'Day':None, 
    'UTC Hour':None,
    'PM2.5':None, 
    'PM10_mask':None,
    'Retrospective':None, 
    'latitude':None,
    'longitude':None
}
"""
hourly_data = {
    'Date':None,
    'UTC Hour':None,
    'PM2.5':None, 
    'latitude':None,
    'longitude':None
}

def read_dust_txt_file(file):  
    pv_detail = province_dict.copy()
    d = hourly_data.copy()
    dust_data = pd.DataFrame()
    #continue parameters or start
    if not os.path.exists(os.path.dirname("Data_Province/")):
        try:
            os.makedirs(os.path.dirname("Data_Province/"))
            print("Create : Data_Province   (Done)")
        
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    line_list = file.split("\n")
    for line in line_list[0:9]:
        col,value = line.split(":")
        pv_detail[col[2:]] = value.strip()
        print(line)

    d = hourly_data.copy()
    d['latitude'] = pv_detail['Latitude']
    d['longitude'] = pv_detail['Longitude']
    for line in line_list[10:-1]:
        #print(line)
        (d['Year'],d['Month'],d['Day'],d['UTC Hour'],pm25,  
        d['PM10_mask'],d['Retrospective']) = line.split("\t")
        d['Date'] = datetime(int(d['Year']),int(d['Month']),int(d['Day']))
        d['PM2.5'] = float(pm25)
        dust_data = dust_data.append(d,ignore_index=True)
   
    return pv_detail , dust_data


In [43]:
data_all_province = pd.DataFrame();
data_all_perday =  pd.DataFrame();
province_list = ['Chiang_Rai','Chiang_Mai','Lampang','Bangkok']
for province in province_list:
    p = province_dict.copy()
    url2 = f'http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/{province}/{province}.txt' #แสดงค่าที่ได้จากจุดตรวจวัดล่าสุด
    params = { }
    response = requests.request('GET', url2 ,params=params)
    in_txt = response.text.split("\n") 
    (p,hour_data) = read_dust_txt_file(response.text)
    
    hour_data.to_csv('Data_Province/'+province + '_data.csv', index=False)
    data_all_province = data_all_province.append(p, ignore_index=True)
    print('='*30)
    
    data_all = hour_data.groupby(['Date','latitude','longitude']).mean()
    data_all_perday = data_all_perday.append(data_all)
    
data_all_perday.to_csv('Data_Province/'+province + '_data_perday.csv')
data_all_province.to_csv('Data_Province/data_all_province.csv', index=False)

% Country: Thailand
% City: Chiang Rai
% City (ASCII): Chiang Rai
% Region: Chiang Rai
% Region (ASCII): Chiang Rai
% Population: 78756
% Latitude: 19.9086
% Longitude: 99.8325
% Time Zone: Asia/Bangkok
% Country: Thailand
% City: Chiang Mai
% City (ASCII): Chiang Mai
% Region: Chiang Mai
% Region (ASCII): Chiang Mai
% Population: 200952
% Latitude: 18.7904
% Longitude: 98.9847
% Time Zone: Asia/Bangkok
% Country: Thailand
% City: Lampang
% City (ASCII): Lampang
% Region: Lampang
% Region (ASCII): Lampang
% Population: 156139
% Latitude: 18.2923
% Longitude: 99.4928
% Time Zone: Asia/Bangkok
% Country: Thailand
% City: Bangkok
% City (ASCII): Bangkok
% Region: Bangkok
% Region (ASCII): Bangkok
% Population: 5104476
% Latitude: 13.754
% Longitude: 100.5014
% Time Zone: Asia/Bangkok


In [44]:
data_all_perday.head(10)

,,,PM2.5
Date,latitude,longitude,
2016-03-03,19.9086,99.8325,46.506667
2016-03-04,19.9086,99.8325,37.766667
2016-03-05,19.9086,99.8325,40.004167
2016-03-06,19.9086,99.8325,50.183333
2016-03-07,19.9086,99.8325,51.937500
2016-03-08,19.9086,99.8325,51.829167
2016-03-09,19.9086,99.8325,51.304167
2016-03-10,19.9086,99.8325,64.791304
2016-03-11,19.9086,99.8325,59.216667
